# Get old tweets using the academic research endpoint through Twitter's API

Get large trunck of tweets through Twitter's API: all \#china tweets posted during the year of 2020 and 2021 quarter by quarter (excl. retweets/quotes/reply).

_Author: Jinghua Xu_

_reference: [https://towardsdatascience.com/an-extensive-guide-to-collecting-tweets-from-twitter-api-v2-for-academic-research-using-python-3-518fcb71df2a](https://towardsdatascience.com/an-extensive-guide-to-collecting-tweets-from-twitter-api-v2-for-academic-research-using-python-3-518fcb71df2a)_

_[Twitter developer portal](https://developer.twitter.com/en/portal/dashboard)_

In [65]:
# For sending GET requests from the API
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time

In [66]:
os.environ['TOKEN'] = 'AAAAAAAAAAAAAAAAAAAAAIE%2BZgEAAAAAf5nHOqUwHaylmZeOihu%2FIqfrsL8%3DEKzW6lM4XxrzGEW3kvS5AL5l0esZvTX6fC5ACEW9Y85xRZ2jDD'

In [67]:
def auth():
    return os.getenv('TOKEN')

In [68]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [69]:

def create_url(keyword, start_date, end_date):
    
    search_url = "https://api.twitter.com/2/tweets/search/all" #Change to the endpoint you want to collect data from

    #change params based on the endpoint you are using
    query_params = {'query': keyword,
                    'start_time': start_date,
                    'end_time': end_date,
                    'max_results': 500,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
    return (search_url, query_params)

In [70]:
def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [71]:
#Inputs for the request
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "#china lang:en -is:retweet -is:reply -is:quote"
start_time = "2021-10-01T00:00:00.000Z"
end_time = "2022-01-01T00:00:00.000Z"

In [72]:
def append_to_csv(json_response, fileName):

    #A counter variable
    counter = 0

    #Open OR create the target CSV file
    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #Loop through each tweet
    for tweet in json_response['data']:
        
        # We will create a variable for each since some of the keys might not exist for some tweets
        # So we will account for that

        # 1. Author ID
        author_id = tweet['author_id']

        # 2. Time created
        created_at = dateutil.parser.parse(tweet['created_at'])

        # # 3. Geolocation
        # if ('geo' in tweet):   
        #     if 'place_id' in tweet['geo']:
        #       geo = tweet['geo']['place_id']
        #     else:
        #       geo = " "
        # else:
        #     geo = " "

        # 4. Tweet ID
        tweet_id = tweet['id']

        # 5. Language
        lang = tweet['lang']

        # 6. Tweet metrics
        retweet_count = tweet['public_metrics']['retweet_count']
        reply_count = tweet['public_metrics']['reply_count']
        like_count = tweet['public_metrics']['like_count']
        quote_count = tweet['public_metrics']['quote_count']

        # 7. source
        source = tweet['source']

        # 8. Tweet text
        text = tweet['text']
        
        # Assemble all data in a list
        res = [author_id, created_at, tweet_id, lang, like_count, quote_count, reply_count, retweet_count, source, text]
        
        # Append the result to the CSV file
        csvWriter.writerow(res)
        counter += 1

    # When done, close the CSV file
    csvFile.close()

    # Print the number of tweets for this iteration
    print("# of Tweets added from this response: ", counter) 

In [73]:
#Total number of tweets we collected from the loop
total_tweets = 0

# output file name
fn = "2021_Q4.csv"

# Create file
csvFile = open(fn, "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['author id', 'created_at', 'id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count','source','tweet'])
csvFile.close()

# Inputs
count = 0 # Counting tweets per time period
# max_count = 100 # Max tweets per time period
flag = True
next_token = None
max_results = 500

# Check if flag is true
while flag:
    print("-------------------")
    print("Token: ", next_token)
    url = create_url(keyword, start_time,end_time)
    json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
    result_count = json_response['meta']['result_count']

    if 'next_token' in json_response['meta']:
        # Save the token to use for next call
        next_token = json_response['meta']['next_token']
        print("Next Token: ", next_token)
        if result_count is not None and result_count > 0 and next_token is not None:
            print("Start Date: ", start_time)
            append_to_csv(json_response, fn)
            count += result_count
            total_tweets += result_count
            print("Total # of Tweets added: ", total_tweets)
            print("-------------------")
            time.sleep(5)                
    # If no next token exists
    else:
        if result_count is not None and result_count > 0:
            print("-------------------")
            print("Start Date: ", start_time)
            append_to_csv(json_response, fn)
            count += result_count
            total_tweets += result_count
            print("Total # of Tweets added: ", total_tweets)
            print("-------------------")
            time.sleep(5)
        
        #Since this is the final request, turn flag to false to move to the next time period.
        flag = False
        next_token = None
    time.sleep(5)
print("Total number of results: ", total_tweets)

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpe17viuqkp8ppnp0r05cptm2ae265
Start Date:  2021-10-01T00:00:00.000Z
# of Tweets added from this response:  493
Total # of Tweets added:  493
-------------------
-------------------
Token:  b26v89c19zqg8o3fpe17viuqkp8ppnp0r05cptm2ae265
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpe17visxsnnd1jksf891wah9pjvy5
Start Date:  2021-10-01T00:00:00.000Z
# of Tweets added from this response:  499
Total # of Tweets added:  992
-------------------
-------------------
Token:  b26v89c19zqg8o3fpe17visxsnnd1jksf891wah9pjvy5
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpe17vgppjxxw045dtg57itdmlrv5p
Start Date:  2021-10-01T00:00:00.000Z
# of Tweets added from this response:  497
Total # of Tweets added:  1489
-------------------
-------------------
Token:  b26v89c19zqg8o3fpe17vgppjxxw045dtg57itdmlrv5p
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpe17vem6rwt1lzc7v5vm40c3v9w1p
Sta

## Data Overview

In [74]:
df = pd.read_csv(fn)
df

,author id,created_at,id,lang,like_count,quote_count,reply_count,retweet_count,source,tweet
0,49919826,2021-12-31 23:58:03+00:00,1477066552839004160,en,0,0,0,0,Instagram,Happy new year! \n#china #beijing #winter #eve...
1,38971530,2021-12-31 23:57:59+00:00,1477066538331086860,en,0,0,0,0,Paper.li,The latest The Political Science Daily! https:...
2,58010461,2021-12-31 23:57:37+00:00,1477066442705195008,en,0,0,0,0,Twitter Web App,Do you support this? Use your purchasing power...
3,1465236285526880260,2021-12-31 23:57:03+00:00,1477066301612957699,en,0,1,0,0,Twitter for Android,Thoughts?\n\n@beverleyturner @MaajidNawaz\n@Un...
4,783046681,2021-12-31 23:56:45+00:00,1477066228028133380,en,1,0,2,1,Twitter Web App,#Putin Tells #Biden &amp; #EU Cut #NATO Crap &...
...,...,...,...,...,...,...,...,...,...,...
241239,758341474119520256,2021-10-01 00:00:26+00:00,1443727470197346309,en,0,0,0,0,WordPress.com,China's electric carmakers are darting into Eu...
241240,50611261,2021-10-01 00:00:05+00:00,1443727380909023233,en,1,0,0,0,Oktopost,🇨🇳 News story - Argus' #copper concentrate TC/...
241241,3508175477,2021-10-01 00:00:01+00:00,1443727366346485764,en,19,3,2,9,Twitter Web App,Today we mark the 72nd anniversary of the foun...
241242,1219090523811528705,2021-10-01 00:00:00+00:00,1443727360621101072,en,13,4,1,21,Twitter Web App,🇨🇳🎉Today marks the 72nd anniversary of the fou...


In [75]:
len(df)

241244

In [76]:
from google.colab import files
files.download(fn)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>